## Load Packages

In [1]:
#using Dates, DelimitedFiles  #in Julia 0.7

include("printmat.jl")

printmatDate (generic function with 8 methods)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Portfolio Choice with Background Risk

Recall: the investor maximizes
$\text{E}R_{p}-\frac{k}{2}\text{Var}(R_{p})$, where 

$R_{p}=v^{\prime}R+\phi R_{H}+(1-\mathbf{1}^{\prime}v-\phi)R_{f}$

Also, $\text{Var}(R_{p}) = u'\Sigma u$, where $u = [v;\phi]$

Notice that with $\sigma_{1H}=0$, the choice of $v$ is not affected by $\phi$ (in spite of the fact that the background risk affects utility).

In [3]:
function EU(v,ϕ,k,μe,Σ,μeH=0)  #expected utility for the case with background risk
    u     = [v;ϕ]              #portfolio weight vector, all risky assets. Background risk last.       
    ERp   = u'*[μe;μeH]        #μeH defaults to zero
    VarRp = u'Σ*u
    EUtil = ERp - k/2*VarRp
    return EUtil
end

EU (generic function with 2 methods)

In [4]:
Σ_A = [0.04 0;                 #covariance with H is zero
       0    0.01]              #σHH does not matter for the choice, just set it to 0.01  
Σ_C = [0.04 0.01;              #alternative covariance matrix: covariance with H is positive
       0.01 0.01]       

v = linspace(0.4,1,51)
L = length(v)

EUM = fill(NaN,(L,3))                   #try different v values (weight on risky investable asset)
for i = 1:L
    EUM[i,1] = EU(v[i],0.0,3,0.08,Σ_A)
    EUM[i,2] = EU(v[i],0.5,3,0.08,Σ_A)    
    EUM[i,3] = EU(v[i],0.5,3,0.08,Σ_C)
end    

In [6]:
println("Notice: with σ1H=0, the choice of v is not affected by ϕ")

plot(v,EUM[:,1],color=:red,linewidth=2,label="phi=0")
plot!(v,EUM[:,2],color=:blue,linewidth=2,label="phi=0.5, sigma1H=0 ")
plot!(v,EUM[:,3],color=:green,linewidth=2,label="phi=0.5, sigma1H=0.01    ")
plot!(legend=:bottomleft)
title!("Expected utility")
xlabel!("v, weight on risky asset")

Notice: with σ1H=0, the choice of v is not affected by ϕ


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 0.025 
 
 
 Expected utility 
 
 
 v, weight on risky asset 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,86.648 56.0996,81.7956 66.7979,77.1665 77.4962,72.7608 88.1945,68.5784 98.8927,64.6195 109.591,60.884 120.289,57.3718 130.988,54.0831 141.686,51.0177 
 152.384,48.1757 163.082,45.5571 173.781,43.1619 184.479,40.99 195.177,39.0416 205.875,37.3166 216.574,35.8149 227.272,34.5366 237.97,33.4817 248.669,32.6502 
 259.367,32.0421 270.065,31.6574 280.763,31.4961 291.462,31.5581 302.16,31.8436 312.858,32.3524 323.556,33.0846 334.255,34.0402 344.953,35.2192 355.651,36.6216 
 366.35,38.2473 377.048,40.0965 387.746,42.169 398.444,44.465 409.143,46.9843 419.841,49.727 430.539,52.6931 441.237,55.8826 451.936,59.2954 462.634,62.9317 
 473.332,66.7913 484.031,70.8744 494.729,75.1808 505.427,79.7106 516.125,84.4638 526.824,89.4404 537.522,94.6404 548.22,100.064 558.918,105.71 569.617,111.581 
 580.315,117.674 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,135.126 56.0996,130.274 66.7979,125.645 77.4962,121.239 88.1945,117.057 98.8927,113.098 109.591,109.362 120.289,105.85 130.988,102.561 141.686,99.4959 
 152.384,96.6539 163.082,94.0353 173.781,91.6401 184.479,89.4683 195.177,87.5199 205.875,85.7948 216.574,84.2932 227.272,83.0149 237.97,81.96 248.669,81.1285 
 259.367,80.5204 270.065,80.1357 280.763,79.9743 291.462,80.0364 302.16,80.3218 312.858,80.8306 323.556,81.5629 334.255,82.5185 344.953,83.6975 355.651,85.0998 
 366.35,86.7256 377.048,88.5748 387.746,90.6473 398.444,92.9432 409.143,95.4625 419.841,98.2053 430.539,101.171 441.237,104.361 451.936,107.774 462.634,111.41 
 473.332,115.27 484.031,119.353 494.729,123.659 505.427,128.189 516.125,132.942 526.824,137.919 537.522,143.119 548.22,148.542 558.918,154.189 569.617,160.059 
 580.315,166.152 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,212.692 56.0996,210.166 66.7979,207.864 77.4962,205.785 88.1945,203.93 98.8927,202.298 109.591,200.889 120.289,199.704 130.988,198.742 141.686,198.004 
 152.384,197.489 163.082,197.197 173.781,197.129 184.479,197.284 195.177,197.662 205.875,198.264 216.574,199.09 227.272,200.138 237.97,201.41 248.669,202.906 
 259.367,204.625 270.065,206.567 280.763,208.733 291.462,211.122 302.16,213.734 312.858,216.57 323.556,219.629 334.255,222.912 344.953,226.417 355.651,230.147 
 366.35,234.1 377.048,238.276 387.746,242.675 398.444,247.298 409.143,252.144 419.841,257.214 430.539,262.507 441.237,268.023 451.936,273.763 462.634,279.726 
 473.332,285.913 484.031,292.323 494.729,298.956 505.427,305.813 516.125,312.893 526.824,320.197 537.522,327.724 548.22,335.474 558.918,343.448 569.617,351.645 
 580.315,360.065 
 "/>
 
 
 
 
 phi=0 
 
 
 
 phi=0.5, sigma1H=0 
 
 
 
 phi=0.5, sigma1H=0.01

# Asset Pricing Implications of Background Risk I: CAPM

Recall:

beta from CAPM regression: $R_{i}^{e}=\alpha_{i}+\beta_{i}R_{m}^{e}+\varepsilon_{i}$ is 

$\beta_{i}=\frac{\sigma_{im}}{\sigma_{mm}}$

Correct pricing expression:

$\mu_{i}^{e} =\tilde{\beta}_{i}\mu_{m}^{e}$, where 
$\tilde{\beta}_{i}  =\frac{\sigma_{im}+\phi\left(  \sigma_{iH}-\sigma_{im}\right)  }{\sigma_{mm}+\phi\left(\sigma_{mH}-\sigma_{mm}\right)  }$

The $\alpha_{i}$ in the CAPM regression can be shown to be
$\alpha_{i}=(\tilde{\beta}_{i}-\beta_{i})\mu_{m}^{e}$

In [7]:
βtilde(σim,ϕ,σiH,σmm,σmH) = (σim + ϕ*(σiH-σim))/(σmm + ϕ*(σmH-σmm))     #a function giving βtilde as above

βtilde (generic function with 1 method)

In [8]:
σim = 0.8                                             #parameter values for the calculations below
σmm = 1
σiH = -0.5
σmH = 0.5
μem = 0.08

β = σim/σmm
βtildeA = βtilde(σim,0,σiH,σmm,σmH)                   #case A, βtilde when ϕ=0
βtildeB = βtilde(σim,0.3,σiH,σmm,σmH)                 #case B, βtilde when ϕ=0.3, otherwise the same 

println("betas: CAPM and the ones to use in finding μᵢ")
printmat([β βtildeA βtildeB])

println("Expected excess return according to βtilde (μᵉ)")
printmat([βtildeA*μem βtildeB*μem])  

println("alphas in CAPM regressions: no background risk, background risk case A, case B")
αA = (βtildeA - β)*μem
αB = (βtildeB - β)*μem
printmat([0 αA αB])

betas: CAPM and the ones to use in finding μᵢ
     0.800     0.800     0.482

Expected excess return according to βtilde (μᵉ)
     0.064     0.039

alphas in CAPM regressions: no background risk, background risk case A, case B
     0.000     0.000    -0.025



# Asset Pricing Implications of Background Risk II: Multi-Factor Model (extra)

In [9]:
σHH = 2
μeH = 0.089

Σ = [σmm σmH;                   #covariance matrix of [Rm,RH]
     σmH σHH]
SH = [σim;σiH]                  #Cov(Ri,Rm) and Cov(Ri,RH)

βmult = inv(Σ)*SH               #coeffs in Ri = βmult'[Rm;RH]
printlnPs("The multiple regression coefficients: ",βmult)

μei = βmult'*[μem;μeH]
printlnPs("\nμᵉ according to 2-factor model (cf. previous results) ",μei)

The multiple regression coefficients:      1.057    -0.514

μᵉ according to 2-factor model (cf. previous results)      0.039


# Empirical Test of a 3-Factor Model: Loading Data

In [10]:
x    = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                 #no. obs and  no. test assets

(388, 5)

## OLS Estimation and Testing α = 0

Recall: estimate $(\alpha_{i},b_{i})$ in the factor model

$R_{it}^{e}  =\alpha_{i}+b_{i}'f_{t} + \varepsilon_{it}$

Test if $\alpha_{i}=0$

Under the Gauss-Markov assumptions, $\textrm{Cov}([\alpha_{i},b_{i}]) = (X'X)^{-1}\textrm{Var}(\varepsilon_{it})$,
where row $t$ of $X$ contains $[1, f_{t}]$. Use element [1,1] of the covariance matrix as the variance of the $\alpha_i$ estimate

In [11]:
x   = [ones(T) Rme RSMB RHML]         #regressors
Σ_1 = inv(x'x)

α     = fill(NaN,n)
tstat = fill(NaN,n)
for i = 1:n                              #loop over the different test assets
    local b_i, residual, Covb
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    α[i]      = b_i[1]
    Covb      = Σ_1*var(residual)   
    tstat[i] = (α[i]-0)/sqrt(Covb[1,1])
end    
println("\nOLS intercepts and t-stats, regressing Re on constant and 3 factors")
printmat([α tstat])


OLS intercepts and t-stats, regressing Re on constant and 3 factors
    -0.513    -2.306
    -0.006    -0.066
     0.030     0.328
    -0.020    -0.206
    -0.015    -0.133

